In [1]:
import google.generativeai as genai
from google.api_core import retry
import os
import requests
from bs4 import BeautifulSoup
from googlesearch import search
import pdfplumber
from io import BytesIO
from googleapiclient.discovery import build

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])


c:\Users\Vincenzo\miniconda3\envs\ema-gpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from gpt_utils import GPT_PROMPT
from gpt_utils import get_text_from_url, google_search

In [3]:
google_search('What is the deadline to submit an article 46?')

'1: Url: https://www.ema.europa.eu/en/human-regulatory-overview/post-authorisation/paediatric-medicines-post-authorisation/submitting-results-paediatric-studies. Title: Submitting results of paediatric studies | European Medicines Agency. Description: 1 . What is the “Article 46 paediatric study submission”? Rev. Oct 2023.. \n 2: Url: https://www.ncleg.net/EnactedLegislation/Statutes/PDF/ByArticle/Chapter_15a/Article_46.pdf. Title: NC General Statutes - Chapter 15A Article 46. Description: Article 46. ... Article, the investigating law enforcement agency shall provide ... Compensation Act and the deadlines by which the victim must file a claim for.. \n 3: Url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8518420/. Title: Optimizing Pediatric Medicine Developments in the European Union .... Description: within a shorter time frame. Currently, in its implementation of Article 46, the EMA requests that Clinical Study Reports (CSRs) must be submitted within 6\xa0.... \n 4: Url: https://ico

## Set up the model

In this step you collate the functions into a "system" that is passed as `tools`, instantiate the model and start the chat session.

This block includes two options for interacting with the Gemini API. By toggling `use_sys_inst`, you can switch between using Gemini 1.5 Pro with a system instruction (highest quality but free-tier quota may be insufficient for a long chat session) or Gemini 1.0 Pro (higher free quota but does not support system instructions).

A retriable `send_message` function is also defined to help with low-quota conversations.

In [4]:
ordering_system = [google_search, get_text_from_url]

model_name = 'gemini-1.5-flash' 

model = genai.GenerativeModel(
    model_name, tools=ordering_system, system_instruction=GPT_PROMPT)

convo = model.start_chat(enable_automatic_function_calling=True)

@retry.Retry(initial=30)
def send_message(message):
  return convo.send_message(message, stream=False)

[21:52, 06/06/2024] Tram Chuc: https://www.ema.europa.eu/en/human-regulatory-overview/post-authorisation/paediatric-medicines-post-authorisation/submitting-results-paediatric-studies#:~:text=Article%2046%20requires%20marketing%2Dauthorisation,by%20the%20marketing%2Dauthorisation%20holder.
[21:53, 06/06/2024] Tram Chuc: What is the deadline to submit an article 46

In [ ]:
from IPython.display import display, Markdown


answer = send_message("What is the deadline to submit an article 46?")

Markdown(answer.text)